In [2]:
# import os
# os.getcwd()

## Import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry as shp
from random import sample
import requests
import polyline

In [5]:
## EV Project Process Script ----
## Read in all_roads shape file
## EPSG 4326 corresponds to geospatial data across a standardized coordinate system (WGS84) for Earth
## EPSG 2231 is a subset of 4326 specific to regions covering Colorado using UTM vs. Lat/Long
## Re-evaluating EPSG 2231 because it doesn't cover all of Colorado
## EPSG 2232 corresponds to central Colorado - RIP
## EPSG 32613 corresponds to UTM 13N which covers all of Colorado
all_roads = gpd.read_file('/home/jcarey9/ev_chargers/data/all_roads.shp').to_crs("GCS_WGS84")
all_roads_proj = all_roads.to_crs("EPSG:32613")

## Read in chgs shape file
chgs = gpd.read_file('/home/jcarey9/ev_chargers/data/alt_fuel_stations.shp').to_crs("GCS_WGS84")
chgs_proj = chgs.to_crs("EPSG:32613")

## Create initial constants
route_dist = 402336
fuel_dist = route_dist * .25
n_sim = 3

## Get the CRS for future calculations between geometries
wgs = all_roads.crs
epsg = all_roads_proj.crs

## Run simulation
# sim_outcomes = route_process(all_roads, chgs, route_dist, fuel_dist, n_sim)

## Save outcomes to file
# pd.to_csv('/home/jcarey9/ev_chargers/data/ev_sim_outcomes.csv')

In [8]:
## Create sample point function for starting and ending positions ----
# from random import sample
# import geopandas as gpd
# import shapely.geometry as shp

def sample_point(data, crs):
    
    data = all_roads_proj.copy()
    
    ## sample one random row from 'data'
    sample_road_row = sample(range(len(data)), 1)

    ## Filter down 'data' to sample road
    sample_road_data = data.loc[sample_road_row].reset_index(drop = True)

    ## Randomly select a pair of coordinates within the road we've selected as a starting point
    sample_road_coords = list(sample_road_data.geometry[0].coords)
    sample_index = sample(range(len(sample_road_coords)), 1)
    sample_geo = shp.Point(list(sample_road_coords[sample_index[0]]))

    point_df = gpd.GeoDataFrame({'geometry': [sample_geo]}, crs = "EPSG:32613")
    point_df = point_df.to_crs(crs)

    ## Return a list with the corresponding sample road row number in 'data' and a geometry object
    return({
        "road_row_num" : sample_road_data.record[0], 
        "point" : point_df
    })

In [9]:
## Get starting point function ----
def get_start_point(df, crs):
    
    data = df.copy()

    # filter only roadways of level 3
    valid_startpoints = data[data["level"] == 3].reset_index(drop = True)

    return sample_point(valid_startpoints, crs)

In [10]:
## Get ending point function ----
def get_end_point(df, start_pos, route_dist, crs):
    
    ## Testing/CRS translation if given wrong data set
    # data = all_roads_proj
    # start_pos = get_start_point(all_roads, epsg)

    # start_pos['point']

    # if (data.crs.utm_zone == None):
    #     data = data.to_crs("EPSG:32613")
    
    data = df.copy()
    
    sp = start_pos['point'].to_crs("EPSG:32613")
    
    # calculate distances from start_position, these are in meters since we are using EPSG:32613
    data['distances'] = data['geometry'].distance(sp.geometry[0])
    # min(data['distances'])
    
    # filter only distances that are further than half route_dist
    valid_endpoints = data[data['distances'] >= (route_dist / 2)].reset_index(drop = True)
    
    return sample_point(valid_endpoints, crs)

In [11]:
def calculate_route(pickup, dropoff, crs):
    ## Convert pickup coordinates from UTM to CRS specified
    pickup_lon = pickup['point'].to_crs(crs).iloc[0]['geometry'].x
    pickup_lat = pickup['point'].to_crs(crs).iloc[0]['geometry'].y
    
    ## Convert dropoff coordinates from UTM to CRS specified
    dropoff_lon = dropoff['point'].to_crs(crs).iloc[0]['geometry'].x
    dropoff_lat = dropoff['point'].to_crs(crs).iloc[0]['geometry'].y
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc + "?annotations=true") 
    if r.status_code!= 200:
        return {}
  
    res = r.json()
    ## TODO (5/11/22) - the response should have additional fields with data we want to pull, such as distance between points
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

In [12]:
## Route process function ----
# def simulate_trip(road_data, evc_data, route_dist, fuel_dist, n_sim = 1, crs):
    
# for i in list(range(1, n_sim + 1)):
#     print(i)

start_pos = get_start_point(all_roads_proj, wgs)
end_pos = get_end_point(all_roads_proj, start_pos, route_dist, wgs)

# print(start_pos['point'])
# print(end_pos['point'])

route = calculate_route(start_pos, end_pos, wgs)



In [13]:
# print(route['distance'])
route

# route_details_init = gpd.GeoDataFrame({'geometry': [shp.Point(x) for x in route['route']]}, crs = wgs)

# ## Trying to split each data frame at first
# # rd_offset = gpd.GeoDataFrame({'geometry': route_details_init['geometry'].shift(-1)}, crs = wgs)
# # rd_offset['geometry'] = rd_offset.where(rd_offset['geometry'] == None, route_details_init['geometry'], axis = 0)
# # print(rd_offset)

# ## Combined them
# route_details_init['lead'] = route_details_init['geometry'].shift(-1)
# route_details_init['lead'] = route_details_init.where(route_details_init['lead'] == None, route_details_init['geometry'], axis = 0)
# route_details_init['lead'] = gpd.GeoSeries(route_details_init['lead'], crs = wgs)

# print(route_details_init)

# ## To calculate the distances between each of the series I've created (geometry and lead), we would need to switch to a projected CRS
# ## Doing so converts all points in geometry to (Inf, Inf) which is impossible calculate on?
# route_details_init['geometry']

# route_details_init['distances'] = route_details_init['geometry'].distance(route_details_init['lead'], align = False)





{'route': [(38.47348, -107.87557),
  (38.53232, -107.93683),
  (38.69279, -108.01781),
  (38.74028, -108.0773),
  (38.76828, -108.22353),
  (38.81196, -108.2936),
  (38.84233, -108.29845),
  (39.01456, -108.47446),
  (39.10433, -108.4457),
  (39.12057, -108.32437),
  (39.21043, -108.25654),
  (39.23323, -108.27238),
  (39.26048, -108.25877),
  (39.37057, -108.16843),
  (39.47344, -108.02051),
  (39.51992, -107.84799),
  (39.5322, -107.69425),
  (39.57028, -107.5325),
  (39.57492, -107.43956),
  (39.5487, -107.32427),
  (39.56658, -107.27263),
  (39.5574, -107.24375),
  (39.6454, -107.07711),
  (39.65097, -106.85317),
  (39.71186, -106.70045),
  (39.65656, -106.63119),
  (39.60829, -106.45055),
  (39.64498, -106.39053),
  (39.64615, -106.3119),
  (39.59078, -106.2454),
  (39.50713, -106.20294),
  (39.50315, -106.14902),
  (39.5628, -106.12946),
  (39.6279, -106.06867),
  (39.70139, -105.86263),
  (39.69447, -105.72645),
  (39.74302, -105.68284),
  (39.76594, -105.62849),
  (39.74081, -1

In [7]:
test = gpd.GeoSeries([shp.Point(-105.211, 38.962)], crs = wgs)
## TODO (5/11/22) - Points must be passed following (long, lat) not (lat, long) otherwise will return (inf, inf)
test.to_crs(epsg = 32613)


0    POINT (481719.362 4312580.862)
dtype: geometry